In [2]:
import networkx as nx
import dgl
import numpy as np  
import pandas as pd 

/home/wyuhui/.conda/envs/rem/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
path = '../../data/project_170/topo-2/8/matrix/59.csv'

matrix = pd.read_csv(path,index_col=0)

In [15]:
import os
import dgl
import dgl.nn as dglnn
import torch
import torch.nn as nn   
import torch.nn.functional as F
import networkx as nx
import numpy as np  
import pandas as pd 
from tqdm import tqdm
import random

class Params():
    def __init__(self, point_num, device, project_dir, matrix_dir) -> None:
        self.N = point_num # 仿真采样点数
        self.device = device
        self.project_dir = project_dir
        self.matrix_dir = matrix_dir
        pass

    def set_data_params(self, batch_size, train_val_test_ratio):
        self.batch_size = batch_size
        self.train_val_test_ratio = train_val_test_ratio


    def set_model_params(self, loss_fcn, optimizer, epoches):
        self.loss_fcn = loss_fcn
        self.optimizer = optimizer
        self.epoches = epoches
        return


def load_graphs(params: Params):
    positions = pd.read_csv(os.path.join(params.project_dir,'positions.csv'), index_col=0).to_numpy()
    mean_x = sum(positions.T[0]) / positions.shape[0]
    mean_y = sum(positions.T[1]) / positions.shape[0]

    connection_matrix = pd.read_csv(os.path.join(params.project_dir, 'connection_matrix.csv'), index_col=0).to_numpy()

    
    all_graphs = []
    for csv_name in tqdm(os.listdir(params.matrix_dir), desc='loading graphs...'):
        matrix = pd.read_csv(os.path.join(params.matrix_dir, csv_name), index_col=0)
        g_nx = nx.from_numpy_array(matrix.to_numpy())
        nodes_id = matrix.columns.astype(int)
        g = dgl.from_networkx(g_nx)
        g.ndata['node_id'] = torch.tensor(nodes_id)
        g.ndata['position'] = torch.tensor(np.array([positions[i] for i in nodes_id]), dtype=torch.float32) # 位置标签
        g.ndata['feat'] = torch.tensor([[mean_x, mean_y] for _ in range(g.number_of_nodes())], dtype=torch.float32) # 初始化节点特征
        # g.ndata['feat'] = torch.tensor(np.array([positions[i] for i in nodes_id]), dtype=torch.float32) # 位置标签
        
        power_features = []
        # 给边特征赋值 power
        for i in range(g.num_edges()):
            src_node_id = nodes_id[int(g.edges()[0][i])]
            dst_node_id = nodes_id[int(g.edges()[1][i])]
            power_features.append(connection_matrix[src_node_id][dst_node_id])
        g.edata['power'] = torch.tensor(power_features, dtype=torch.float32)
        g = g.int().to(params.device)
        all_graphs.append(g)
    return all_graphs


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'{device} device found')
project_dir = '../../data/project_small_209'

matrix_dir = os.path.join(project_dir, 'topo-1/6/matrix') #!
point_num = 209 #!

map_weight = pd.read_csv(os.path.join(project_dir, 'map_weight.csv'), index_col=0).to_numpy().T
map_weight = torch.tensor(map_weight, dtype=torch.float32).to(device) # shape = (N*N) * 5


params = Params(point_num, device, project_dir, matrix_dir)

graphs = load_graphs(params)

/home/wyuhui/.conda/envs/rem/lib/python3.9/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


cuda device found


loading graphs...: 100%|██████████| 10001/10001 [00:33<00:00, 301.47it/s]


In [16]:
g = graphs[0]

In [18]:
g.ndata

{'node_id': tensor([ 75,  87,  89, 100, 140, 145], device='cuda:0'), 'position': tensor([[3.4613, 1.0340],
        [3.9613, 0.5340],
        [4.4613, 5.0340],
        [4.9613, 5.0340],
        [6.4613, 1.5340],
        [6.9613, 4.5340]], device='cuda:0'), 'feat': tensor([[4.9613, 3.0340],
        [4.9613, 3.0340],
        [4.9613, 3.0340],
        [4.9613, 3.0340],
        [4.9613, 3.0340],
        [4.9613, 3.0340]], device='cuda:0')}